In [ ]:
import yfinance as yf
import pandas as pd
import talib
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
# from fear_and_greed import FearAndGreedIndex

# ============ DOWNLOAD BITCOIN DATA ============
data_start = '2017-01-01'
data_end = '2025-11-05'
bitcoin_historical_data = yf.download('BTC-USD', start=data_start, end=data_end)

# ============ CALCULATE TECHNICAL INDICATORS ============
bitcoin_close_prices = bitcoin_historical_data[('Close', 'BTC-USD')].values
bitcoin_rsi_values = talib.RSI(bitcoin_historical_data[('Close', 'BTC-USD')], timeperiod=14)

# ============ ACCESS OTHER INDICATORS ============

# fear_and_greed_index = FearAndGreedIndex()
# bitcoin_fgi = fear_and_greed_index.get_last_n_days(data_end-data_start)
# print(f"✓ Fear & Greed data downloaded: {len(bitcoin_fgi)} days")

# Calculate MACD line (fast momentum)
bitcoin_macd_line, bitcoin_macd_signal_line, bitcoin_macd_histogram = talib.MACD(
    bitcoin_close_prices, 
    fastperiod=12,      # Fast exponential moving average
    slowperiod=26,      # Slow exponential moving average
    signalperiod=9      # Signal line period
)  # Returns (MACD, signal, histogram) - we want MACD

# Add to dataset
bitcoin_historical_data['RSI'] = bitcoin_rsi_values
bitcoin_historical_data['MACD'] = bitcoin_macd_histogram
bitcoin_historical_data['MACD_Signal'] = bitcoin_macd_signal_line


# ============ CONTROL PARAMETERS (EDIT THESE) ============
# 'absolute' = show original values (e.g., Bitcoin price in USD)
# 'normalised' = scale values to 0-1 range (for comparing different scales)
# 'both' = show normalized AND absolute on separate subplots
indicator_display_config = {
    ('Close', 'BTC-USD'): 'both',      
    'RSI': 'both',
    'MACD': 'both',                       
    'MACD_Signal': 'both'                 
}

# Color assignment for visual clarity (makes each indicator easy to identify)
indicator_color_mapping = {
    ('Close', 'BTC-USD'): 'blue',       # Bitcoin price = blue
    'RSI': 'red',                        # RSI = red
    'MACD': 'green',                     # MACD = green
    'MACD_Signal': 'orange'              # MACD Signal = orange
}

# ============ PREPARE DATA: CREATE NORMALIZED VERSIONS ============
# We will store BOTH the original (absolute) and normalized versions
# of each indicator so we can plot them separately

data_storage = {
    'absolute_values': {},     # Original values (e.g., Bitcoin price in USD)
    'normalised_values': {}    # Values scaled to 0-1 range
}

# MinMaxScaler transforms values to a 0-1 range: (value - min) / (max - min)
minmax_scaler = MinMaxScaler()

# Loop through each indicator and prepare both versions
for indicator_name, display_mode in indicator_display_config.items():
    # ===== Extract the original data for this indicator =====

    original_indicator_values = bitcoin_historical_data[indicator_name].values
    
    # Store the absolute (original) values
    data_storage['absolute_values'][indicator_name] = original_indicator_values
    
    # ===== Create normalized version (0-1 scale) =====
    # First, create an array full of NaN values (same length as our dataset)
    normalized_array = np.full(len(bitcoin_historical_data), np.nan)
    
    # Find which rows have actual data (not NaN/missing)
    valid_data_mask = bitcoin_historical_data[[indicator_name]].notna().values.flatten()
    
    # Extract only the valid (non-NaN) values
    valid_values_only = bitcoin_historical_data.loc[valid_data_mask, [indicator_name]].values
    
    # Apply MinMaxScaler to normalize these values to 0-1 range
    normalized_values_only = minmax_scaler.fit_transform(valid_values_only).flatten()
    
    # Put the normalized values back in their original positions (NaN stays NaN)
    normalized_array[valid_data_mask] = normalized_values_only
    
    # Store the normalized values
    data_storage['normalised_values'][indicator_name] = normalized_array

C:\Users\Harneet Singh\AppData\Local\Temp\ipykernel_16028\997547810.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  bitcoin_historical_data = yf.download('BTC-USD', start=data_start, end=data_end)
[*********************100%***********************]  1 of 1 completed


In [3]:
# ============ CREATE INTERACTIVE PLOT ============

# Create a figure with 2 rows and 1 column
# vertical_spacing controls the gap between subplots
interactive_plot_figure = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Bitcoin Indicators: Absolute Values", 
                    "Bitcoin Indicators: Normalised (0-1 Scale)"),
    vertical_spacing=0.12  # Space between the two plots (0-1 scale, 0.12 = 12%)
)

# ============ POPULATE TOP SUBPLOT WITH ABSOLUTE VALUES ============
# Loop through each indicator we want to plot
for indicator_name, display_mode in indicator_display_config.items():
    
    # Only plot if display_mode is 'absolute' or 'both'
    if display_mode in ['absolute', 'both']:
        
        # Get the color for this indicator from our mapping
        indicator_color = indicator_color_mapping.get(indicator_name, 'black')
        
        # Get the absolute (original) values for this indicator
        absolute_values_for_indicator = data_storage['absolute_values'][indicator_name]
        
        # Add a line trace to the top subplot (row=1, col=1)
        # Scatter with mode='lines' creates a line plot
        interactive_plot_figure.add_trace(
            go.Scatter(
                x=bitcoin_historical_data.index,              # X-axis: dates
                y=absolute_values_for_indicator,              # Y-axis: actual values
                mode='lines',                                  # Draw as a line (not points)
                name=str(indicator_name),                     # Name shown in legend
                line=dict(
                    color=indicator_color,                     # Use the color we defined
                    width=2                                    # Line thickness
                ),
                # Custom hover text: show value with 4 decimal places
                hovertemplate='<b>' + str(indicator_name) + '</b><br>Value: %{y:.4f}<extra></extra>',
                showlegend=True                               # Show in legend
            ),
            row=1, col=1  # Add to top subplot
        )

# ============ POPULATE BOTTOM SUBPLOT WITH NORMALISED VALUES ============
# Same process as above, but for normalised values in the bottom subplot
for indicator_name, display_mode in indicator_display_config.items():
    
    # Only plot if display_mode is 'normalised' or 'both'
    if display_mode in ['normalised', 'both']:
        
        # Get the color for this indicator
        indicator_color = indicator_color_mapping.get(indicator_name, 'black')
        
        # Get the normalised (0-1 scale) values for this indicator
        normalised_values_for_indicator = data_storage['normalised_values'][indicator_name]
        
        # Add a line trace to the bottom subplot (row=2, col=1)
        interactive_plot_figure.add_trace(
            go.Scatter(
                x=bitcoin_historical_data.index,              # X-axis: dates
                y=normalised_values_for_indicator,            # Y-axis: normalised (0-1)
                mode='lines',                                  # Draw as a line
                name=str(indicator_name),                     # Name shown in legend
                line=dict(
                    color=indicator_color,                     # Same color as absolute version
                    width=2,                                   # Same line thickness),
                ),
                # Custom hover text with 4 decimal places
                hovertemplate='<b>' + str(indicator_name) + '</b><br>Normalised: %{y:.4f}<extra></extra>',
                showlegend=True                               # Show in legend
            ),
            row=2, col=1  # Add to bottom subplot
        )

# ============ CONFIGURE PLOT APPEARANCE ============
# These settings control how the entire plot looks

interactive_plot_figure.update_layout(
    title="Bitcoin Risk Analysis: Indicators Comparison",  # Main title
    hovermode='x unified',                                 # When hovering, show all values at that date
    height=900,                                            # Plot height in pixels
    template='plotly_white',                               # Clean white background theme
    font=dict(size=11)                                     # Font size for readability
)

# ============ LABEL THE Y-AXES ============
# Top subplot y-axis (absolute values)
interactive_plot_figure.update_yaxes(
    title_text="<b>Price (USD) / Indicator Value</b>",    # Label for absolute values
    row=1, col=1                                           # Top subplot
)

# Bottom subplot y-axis (normalised values)
interactive_plot_figure.update_yaxes(
    title_text="<b>Normalised Scale (0 to 1)</b>",        # Label for normalised values
    row=2, col=1                                           # Bottom subplot
)

# ============ LABEL THE X-AXIS ============
# X-axis appears at the bottom, so we only need to label row 2
interactive_plot_figure.update_xaxes(
    title_text="<b>Date</b>",                             # Label for x-axis
    row=2, col=1                                           # Bottom subplot
)


interactive_plot_figure.write_html("bitcoin_indicators_plot.html")
import webbrowser
webbrowser.open("bitcoin_indicators_plot.html")

True